In [1]:
import sys
sys.path.append('/home/jh20/narin/physionet/python-classifier-2022')
from helper_code import *

In [35]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument('--train_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/',     help='train feature path')
parser.add_argument('--valid_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/',     help='validation feature path')
parser.add_argument('--output_folder',    default='./output_folder_LCNN',     help='output folder')
parser.add_argument("--mode",default="murmur", type=str, help="mode : murmur/outcome")
parser.add_argument("--lr",default=1e-6, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=6 ,type=int)
parser.add_argument("--minibatchsize_valid", default=4, type=int)
parser.add_argument("--train_num_workers", default=8, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_LCNN', type=str)
parser.add_argument("--logdir", default='./log_hmd_LCNN/', type=str)
parser.add_argument("--model_name", default='hmd_LCNN_model_murmur', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=80, type=int)
parser.add_argument("--gpu", default="0", type=str)




warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [3]:
def load_challenge_model(model_folder, verbose):
    murmur_train_model = torch.load(model_folder + '/' + 'murmur.model
    outcome_train_model = torch.load(model_folder + '/' + 'outcome.model')
    return [murmur_train_model, outcome_train_model]

In [4]:
model = load_challenge_model(args.model_path, verbose=2)

In [5]:
import sys
sys.path.append("./model")

from CNN_outcome import CNN as CNN_outcome
from LCNN_murmur import Wav2Vec2_LCNN as LCNN_murmur

In [ ]:
def read_recording(wav_fn, audio_length= 20, sr= 4000):  #20초 

    recording, frequency = load_wav_file(wav_fn)
    recording = recording * 1.0
    recording = recording[4000:-4000]

    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording

In [ ]:
def read_recording(wav_fn, audio_length= 20, sr= 4000):

    recording, frequency = librosa.load(wav_fn, sr = sr)
    
    recording = recording*1.0
    recording = recording[4000:-4000]


    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording, frequency

In [6]:
import numpy as np, os, sys
# from helper_code import *
# from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [30]:
import random
import torch
import torch.nn.functional as F
import sys
sys.path.append("./model")

from CNN_outcome import CNN as CNN_outcome
from LCNN_murmur import Wav2Vec2_LCNN as LCNN_murmur


def run_challenge_model(model, data, recordings, verbose):
    murmur_model , outcome_model = model

    GPU_NUM = args.gpu # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU

    
    # Define the model
    murmur_classifier = LCNN_murmur("murmur")
    murmur_classifier = murmur_classifier.cuda(device)
    
    outcome_classifier = CNN_outcome()
    outcome_classifier = outcome_classifier.cuda(device)
    
    murmur_classifier.load_state_dict(murmur_model)
    outcome_classifier.load_state_dict(outcome_model)
    
#     wav_files = glob.glob(data_folder + '*.wav')
#     num_wav_files = len(wav_files)
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    
    audio_length=20
    sr =4000
    recordings_list=[]
    for i in range(num_recordings):
        
        recording = recordings[i]
        recording = recording*1.0
        recording = recording[4000:-4000]


        length = audio_length*sr
        if recording.shape[0] <= length:
            shortage = length - recording.shape[0]
            recording = np.pad(recording, (0, shortage), 'wrap')
        start_frame = np.int64(random.random()*(recording.shape[0]-length))
        recording = recording[start_frame:start_frame + length] 
        recordings_list.append(recording)
        

    features = get_features(data, recordings_list)
    
    predict_murmur_arr= np.zeros((num_recordings,))
    predict_outcome_arr= np.zeros((num_recordings,2))
    
    for i in range(num_recordings):
        
        logmel = torch.tensor(features['mel'][i]).cuda()
        age = torch.tensor(features['age']).cuda()
        sex = torch.tensor(features['sex']).cuda()
        hw = torch.tensor(features['hw']).cuda()
        preg = torch.tensor(features['preg']).cuda()
        loc = torch.tensor(features['loc'][i]).cuda()
        raw_feature = torch.tensor(features['raw'][i], dtype=torch.float32).cuda()
        
        logmel = logmel.unsqueeze(0)
        age = age.unsqueeze(0)
        sex = sex.unsqueeze(0)
        hw = hw.unsqueeze(0)
        preg = preg.unsqueeze(0)
        loc = loc.unsqueeze(0)
        raw_feature = raw_feature.unsqueeze(0)
        
        predict_murmur = murmur_classifier(raw_feature)
        predict_outcome = F.softmax(outcome_classifier(logmel,age,sex,hw,preg,loc), dim=1)
        
        predict_murmur_arr[i]= predict_murmur.data.detach().cpu().numpy()
        predict_outcome_arr[i,:] = predict_outcome.data.detach().cpu().numpy()

    
    
    
   
    # Get classifier probabilities.
    idx1 = predict_murmur_arr.argmax(axis=0)
    murmur_probabilities_max = predict_murmur_arr[idx1] 
    murmur_probabilities = np.array([murmur_probabilities_max,0,1-murmur_probabilities_max])
    idx2 = predict_outcome_arr.argmax(axis=0)[0]
    outcome_probabilities = predict_outcome_arr[idx2,]


    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    if murmur_probabilities_max > 0.39:
        murmur_labels[0] = 1
    else:
         murmur_labels[2] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

In [8]:
import glob
patient_data = glob.glob(args.train_data_folder + '*.txt')

In [9]:
data = load_patient_data(patient_data[0])

In [10]:
recordings = load_recordings(args.train_data_folder, data)

In [31]:
run_challenge_model(model, data, recordings, verbose=2)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(['Present', 'Unknown', 'Absent', 'Abnormal', 'Normal'],
 array([0, 0, 1, 1, 0]),
 array([0.36379722, 0.        , 0.63620278, 0.59413636, 0.40586364]))

In [32]:
run_model(args.model_path, args.valid_data_folder, args.output_folder, allow_failures=True, verbose=2)

Loading Challenge model...
Running model on Challenge data...
    1/191...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    2/191...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    3/191...


...

    190/191...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    191/191...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done.


In [20]:
import librosa
import math
import random

def get_features(data, recordings):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    
    
    feature_dict={}
    
    feature_dict['raw']=[]
    for i in range(num_recordings):
        raw_data = recordings[i]
        feature_dict['raw'].append(raw_data)
        
    feature_dict['mel']=[]
    for i in range(num_recordings):
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = (recordings[i]/32768).astype(np.float32),
                                                         sr= 4000,
                                                         n_mels=128,
                                                         n_fft=400, 
                                                         hop_length=128, 
                                                         win_length=400))
        feature_dict['mel'].append(log_mel_feature)
        
        
        
        
#     recording_info_lst = data.split('\n')[1:num_recordings+1]
#     feature_dict['wav2vec2'] = []
#     for i in range(num_recordings):
#         pk_feature_fnm = recording_info_lst[i].split(' ')[2].replace('wav','pickle')
#         with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/'+ pk_feature_fnm ,'rb') as fr:
#             feature = pickle.load(fr)
#         feature_dict['wav2vec2'].append(feature)
    

    # age
    current_patient_age = get_age(data)
    current_age_group = np.zeros(6, dtype=np.float32)
    if current_patient_age in age_classes:
        j = age_classes.index(current_patient_age)
        current_age_group[j] = 1.0
    else :
        current_age_group[5] = 1.0

    feature_dict['age']=current_age_group



    # sex
    sex = get_sex(data)
    sex_feature = np.zeros(2, dtype=np.float32)
    if compare_strings(sex, 'Female'):
        sex_feature[0] = 1.0
    elif compare_strings(sex, 'Male'):
        sex_feature[1] = 1.0

    feature_dict['sex']=sex_feature

    # height and weight.
    height = get_height(data)
    weight = get_weight(data)

    ## simple impute
    if math.isnan(height) :
        height = 110.846  #mean
    if math.isnan(weight) :
        weight = 23.767   #mean

    height_weight = np.array([height, weight], dtype=np.float32)


    feature_dict['hw']=height_weight

    # Extract pregnancy
    preg_feature = np.zeros(2, dtype=np.float32)
    is_pregnant = get_pregnancy_status(data)
    if is_pregnant == True:
        preg_feature[0] = 1.0
    elif is_pregnant == False:
        preg_feature[1] = 1.0

    feature_dict['preg']=preg_feature

    # Extract location
    feature_dict['loc']=[]
    locations = get_locations(data)
    for j in range(num_recordings):
        
        num_recording_locations = len(recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if locations[j] in recording_locations:
            idx = recording_locations.index(locations[j])
            loc_feature[idx] = 1.0
            
        feature_dict['loc'].append(loc_feature)




#     # label

#     current_murmur = np.zeros(num_murmur_classes, dtype=np.float32)
#     murmur = get_murmur(data)
#     if murmur in murmur_classes:
#         j = murmur_classes.index(murmur)
#         current_murmur[j] = 1
    
#     feature_dict['murmur']=current_murmur
    
#     current_outcome = np.zeros(num_outcome_classes, dtype=np.float32)
#     outcome = get_outcome(data)
#     if outcome in outcome_classes:
#         j = outcome_classes.index(outcome)
#         current_outcome[j] = 1

#     feature_dict['outcome']=current_outcome
    
    
    
    
#     #label_2
    
#     if get_murmur(data) == self.murmur_classes[0]:
#         murmur_lable.append(1)
#     elif get_murmur(current_patient_data) == self.murmur_classes[1]:
#         murmur_lable.append(0)
#     else :
#         murmur_lable.append(0)
    
#     if get_outcome(current_patient_data) == self.outcome_classes[0]:
#         outcome_lable.append(1)
#     elif get_outcome(current_patient_data) == self.outcome_classes[1]:
#         outcome_lable.append(0)
    
    
    


    return feature_dict

In [33]:
# Find Challenge files.
def find_challenge_files(label_folder, output_folder):
    label_files = list()
    output_files = list()
    for label_file in sorted(os.listdir(label_folder)):
        label_file_path = os.path.join(label_folder, label_file) # Full path for label file
        if os.path.isfile(label_file_path) and label_file.lower().endswith('.txt') and not label_file.lower().startswith('.'):
            root, ext = os.path.splitext(label_file)
            output_file = root + '.csv'
            output_file_path = os.path.join(output_folder, output_file) # Full path for corresponding output file
            if os.path.isfile(output_file_path):
                label_files.append(label_file_path)
                output_files.append(output_file_path)
            else:
                raise IOError('Output file {} not found for label file {}.'.format(output_file, label_file))

    if label_files and output_files:
        return label_files, output_files
    else:
        raise IOError('No label or output files found.')

# Load murmurs from label files.
def load_murmurs(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_murmur(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outcomes from label files.
def load_outcomes(label_files, classes):
    num_patients = len(label_files)
    num_classes = len(classes)

    # Use one-hot encoding for the labels.
    labels = np.zeros((num_patients, num_classes), dtype=np.bool_)

    # Iterate over the patients.
    for i in range(num_patients):
        data = load_patient_data(label_files[i])
        label = get_outcome(data)
        for j, x in enumerate(classes):
            if compare_strings(label, x):
                labels[i, j] = 1

    return labels

# Load outputs from output files.
def load_classifier_outputs(output_files, classes):
    # The outputs should have the following form:
    #
    # #Record ID
    # class_1, class_2, class_3
    #       0,       1,       1
    #    0.12,    0.34,    0.56
    #
    num_patients = len(output_files)
    num_classes = len(classes)

    # Use one-hot encoding for the outputs.
    binary_outputs = np.zeros((num_patients, num_classes), dtype=np.bool_)
    scalar_outputs = np.zeros((num_patients, num_classes), dtype=np.float64)

    # Iterate over the patients.
    for i in range(num_patients):
        patient_id, patient_classes, patient_binary_outputs, patient_scalar_outputs = load_challenge_outputs(output_files[i])

        # Allow for unordered or reordered classes.
        for j, x in enumerate(classes):
            for k, y in enumerate(patient_classes):
                if compare_strings(x, y):
                    binary_outputs[i, j] = patient_binary_outputs[k]
                    scalar_outputs[i, j] = patient_scalar_outputs[k]

    return binary_outputs, scalar_outputs

# For each patient, set a specific class to positive if no class is positive or multiple classes are positive.
def enforce_positives(outputs, classes, positive_class):
    num_patients, num_classes = np.shape(outputs)
    j = classes.index(positive_class)

    for i in range(num_patients):
        if np.sum(outputs[i, :]) != 1:
            outputs[i, :] = 0
            outputs[i, j] = 1
    return outputs

# Compute macro AUROC and macro AUPRC.
def compute_auc(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    # Compute and summarize the confusion matrices for each class across at distinct output values.
    auroc = np.zeros(num_classes)
    auprc = np.zeros(num_classes)

    for k in range(num_classes):
        # We only need to compute TPs, FPs, FNs, and TNs at distinct output values.
        thresholds = np.unique(outputs[:, k])
        thresholds = np.append(thresholds, thresholds[-1]+1)
        thresholds = thresholds[::-1]
        num_thresholds = len(thresholds)

        # Initialize the TPs, FPs, FNs, and TNs.
        tp = np.zeros(num_thresholds)
        fp = np.zeros(num_thresholds)
        fn = np.zeros(num_thresholds)
        tn = np.zeros(num_thresholds)
        fn[0] = np.sum(labels[:, k] == 1)
        tn[0] = np.sum(labels[:, k] == 0)

        # Find the indices that result in sorted output values.
        idx = np.argsort(outputs[:, k])[::-1]

        # Compute the TPs, FPs, FNs, and TNs for class k across thresholds.
        i = 0
        for j in range(1, num_thresholds):
            # Initialize TPs, FPs, FNs, and TNs using values at previous threshold.
            tp[j] = tp[j-1]
            fp[j] = fp[j-1]
            fn[j] = fn[j-1]
            tn[j] = tn[j-1]

            # Update the TPs, FPs, FNs, and TNs at i-th output value.
            while i < num_patients and outputs[idx[i], k] >= thresholds[j]:
                if labels[idx[i], k]:
                    tp[j] += 1
                    fn[j] -= 1
                else:
                    fp[j] += 1
                    tn[j] -= 1
                i += 1

        # Summarize the TPs, FPs, FNs, and TNs for class k.
        tpr = np.zeros(num_thresholds)
        tnr = np.zeros(num_thresholds)
        ppv = np.zeros(num_thresholds)
        for j in range(num_thresholds):
            if tp[j] + fn[j]:
                tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
            else:
                tpr[j] = float('nan')
            if fp[j] + tn[j]:
                tnr[j] = float(tn[j]) / float(fp[j] + tn[j])
            else:
                tnr[j] = float('nan')
            if tp[j] + fp[j]:
                ppv[j] = float(tp[j]) / float(tp[j] + fp[j])
            else:
                ppv[j] = float('nan')

        # Compute AUROC as the area under a piecewise linear function with TPR/
        # sensitivity (x-axis) and TNR/specificity (y-axis) and AUPRC as the area
        # under a piecewise constant with TPR/recall (x-axis) and PPV/precision
        # (y-axis) for class k.
        for j in range(num_thresholds-1):
            auroc[k] += 0.5 * (tpr[j+1] - tpr[j]) * (tnr[j+1] + tnr[j])
            auprc[k] += (tpr[j+1] - tpr[j]) * ppv[j+1]

    # Compute macro AUROC and macro AUPRC across classes.
    if np.any(np.isfinite(auroc)):
        macro_auroc = np.nanmean(auroc)
    else:
        macro_auroc = float('nan')
    if np.any(np.isfinite(auprc)):
        macro_auprc = np.nanmean(auprc)
    else:
        macro_auprc = float('nan')

    return macro_auroc, macro_auprc, auroc, auprc

# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

# Define total cost for algorithmic prescreening of m patients.
def cost_algorithm(m):
    return 10*m

# Define total cost for expert screening of m patients out of a total of n total patients.
def cost_expert(m, n):
    return (25 + 397*(m/n) -1718*(m/n)**2 + 11296*(m/n)**4) * n

# Define total cost for treatment of m patients.
def cost_treatment(m):
    return 10000*m

# Define total cost for missed/late treatement of m patients.
def cost_error(m):
    return 50000*m

# Compute Challenge cost metric.
def compute_cost(labels, outputs, label_classes, output_classes):
    # Define positive and negative classes for referral and treatment.
    positive_classes = ['Present', 'Unknown', 'Abnormal']
    negative_classes = ['Absent', 'Normal']

    # Compute confusion matrix.
    A = compute_confusion_matrix(labels, outputs)

    # Identify positive and negative classes for referral.
    idx_label_positive = [i for i, x in enumerate(label_classes) if x in positive_classes]
    idx_label_negative = [i for i, x in enumerate(label_classes) if x in negative_classes]
    idx_output_positive = [i for i, x in enumerate(output_classes) if x in positive_classes]
    idx_output_negative = [i for i, x in enumerate(output_classes) if x in negative_classes]

    # Identify true positives, false positives, false negatives, and true negatives.
    tp = np.sum(A[np.ix_(idx_output_positive, idx_label_positive)])
    fp = np.sum(A[np.ix_(idx_output_positive, idx_label_negative)])
    fn = np.sum(A[np.ix_(idx_output_negative, idx_label_positive)])
    tn = np.sum(A[np.ix_(idx_output_negative, idx_label_negative)])
    total_patients = tp + fp + fn + tn

    # Compute total cost for all patients.
    total_cost = cost_algorithm(total_patients) \
        + cost_expert(tp + fp, total_patients) \
        + cost_treatment(tp) \
        + cost_error(fn)

    # Compute mean cost per patient.
    if total_patients > 0:
        mean_cost = total_cost / total_patients
    else:
        mean_cost = float('nan')

    return mean_cost




In [34]:
def evaluate_model(label_folder, output_folder):
    # Define murmur and outcome classes.
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']

    # Load and parse label and model output files.
    label_files, output_files = find_challenge_files(label_folder, output_folder)
    murmur_labels = load_murmurs(label_files, murmur_classes)
    murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
    outcome_labels = load_outcomes(label_files, outcome_classes)
    outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)

    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')

    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
        murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
        outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)

    # Return the results.
    return murmur_scores, outcome_scores

In [36]:
murmur_scores, outcome_scores = evaluate_model(args.valid_data_folder, args.output_folder)

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string


print(output_string)

with open('./output_score/score,txt', 'a+') as f:
    f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.712,0.518,0.504,0.770,0.752,16338.628

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.586,0.591,0.464,0.560,0.828,12719.618

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.864,0.500,0.771
AUPRC,0.649,0.073,0.833
F-measure,0.660,0.000,0.851
Accuracy,0.868,0.000,0.820

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.586,0.586
AUPRC,0.571,0.610
F-measure,0.691,0.236
Accuracy,0.959,0.140

